Exploring states of the United States that produce sugar in relationship to the percentage of obese adults in corresponding states. 

In [1]:
import pandas as pd 
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
# read in excel file 
excelFile = pd.ExcelFile('../data/week_5_6/Ag_Census_Map_data_07172015.xlsx')
# parse through multiple sheets of excel 
agri_df = {}
for sheet_name in excelFile.sheet_names:
    agri_df[sheet_name] = excelFile.parse(sheet_name)
    

In [3]:
obese = pd.read_csv('../data/week_5_6/Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System.csv')

## Cleaning 

In [71]:
# filter obesity data to include data from 2012 and answers percent value of obese adults 
# total adult obesity per state in 2012 
obese_sub = obese[(obese['YearStart']==2012) & 
      (obese['Question'] == 'Percent of adults aged 18 years and older who have obesity') &
      ~(obese['LocationAbbr'].isin(['US', 'DC'])) &
      (obese['StratificationCategory1'] == 'Total')][['LocationDesc', 'Data_Value']]


In [79]:
obese_sub.rename(columns={'LocationDesc':'StateName', 'Data_Value':'total_adult_obesity_percent'}, inplace=True)

In [40]:
geo_agri = agri_df['County Names'][['FIPSTEXT', 'StateName']]

In [31]:
var_lookup = agri_df['Variable Lookup']
sugar_mid = (var_lookup[var_lookup['MAPTITLE'].isin(['Acres of Sugarbeets for Sugar Harvested as Percent of Harvested Cropland Acreage:  2012',
                                                        'Acres of Sugarcane for Sugar Harvested as Percent of Harvested Cropland Acreage:  2012'])]['MapID']).values




In [32]:
sugar_mid

array(['y12_M328', 'y12_M329'], dtype=object)

In [37]:
sugar = agri_df['Crops and Plants'][['FIPSTEXT','y12_M328_valueNumeric', 'y12_M329_valueNumeric']]

In [42]:
# merge and remove NA values for State 
sugar = pd.merge(sugar, geo_agri, on="FIPSTEXT", how="left")
sugar = sugar[~sugar['StateName'].isna()]

In [47]:
# fill in NA values with zero 
sugar.fillna(0, inplace=True)

/Users/Nina/bin/anaconda3/envs/msds622/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [75]:
# aggregating and renaming 
sugar_agg = sugar.copy()
sugar_agg.drop(columns="FIPSTEXT", inplace=True)
sugar_agg = sugar_agg.groupby(by='StateName').sum().reset_index()
sugar_agg.rename(columns={'y12_M328_valueNumeric': 'sum_acre_sugar_sugarbeets',
                          'y12_M329_valueNumeric':'sum_acre_sugar_sugarcane'}, inplace=True)

In [83]:
sugar_agg['combined_sugar_acres'] = sugar_agg['sum_acre_sugar_sugarbeets'] + sugar_agg['sum_acre_sugar_sugarcane']

In [84]:
sugar_agg

,StateName,sum_acre_sugar_sugarbeets,sum_acre_sugar_sugarcane,combined_sugar_acres
0,Alabama,0.00,0.00,0.00
1,Alaska,0.00,0.00,0.00
2,Arizona,0.00,0.00,0.00
3,Arkansas,0.00,0.00,0.00
4,California,0.00,0.00,0.00
5,Colorado,13.30,0.00,13.30
6,Connecticut,0.00,0.00,0.00
7,Delaware,0.00,0.00,0.00
8,Florida,0.00,187.66,187.66
9,Georgia,0.00,0.00,0.00


In [85]:
# joining sugar and obesity data 
sug_ob = pd.merge(sugar_agg, obese_sub, on="StateName", how="inner")

In [89]:
ob = sug_ob['total_adult_obesity_percent']

In [90]:
sorted(ob)

[20.5,
 22.9,
 23.6,
 23.6,
 23.7,
 24.3,
 24.3,
 24.6,
 24.6,
 25.0,
 25.2,
 25.6,
 25.7,
 25.7,
 25.7,
 26.0,
 26.2,
 26.8,
 26.8,
 26.9,
 27.1,
 27.3,
 27.3,
 27.4,
 27.6,
 28.1,
 28.1,
 28.3,
 28.6,
 29.1,
 29.1,
 29.2,
 29.6,
 29.6,
 29.7,
 29.7,
 29.8,
 30.1,
 30.4,
 31.1,
 31.1,
 31.3,
 31.4,
 31.6,
 32.2,
 33.0,
 33.8,
 34.5,
 34.6]

In [91]:
sug_ob

,StateName,sum_acre_sugar_sugarbeets,sum_acre_sugar_sugarcane,combined_sugar_acres,total_adult_obesity_percent
0,Alabama,0.00,0.00,0.00,33.0
1,Alaska,0.00,0.00,0.00,25.7
2,Arizona,0.00,0.00,0.00,26.0
3,Arkansas,0.00,0.00,0.00,34.5
4,California,0.00,0.00,0.00,25.0
5,Colorado,13.30,0.00,13.30,20.5
6,Connecticut,0.00,0.00,0.00,25.6
7,Delaware,0.00,0.00,0.00,26.9
8,Florida,0.00,187.66,187.66,25.2
9,Georgia,0.00,0.00,0.00,29.1


In [87]:
sug_ob.to_csv('sugar_obesity_2012.csv', index=False)